In [1]:
import csv
import ast
import numpy as np
import time

start = time.time()

# --- Load BB (Billboard data) ---
BB = []
with open("BB_LA.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        BB.append({
            'B_index': int(row['B_index']),           
            'Influence': float(row['Influence'])     
        })

# Sort billboard data by descending influence
BB.sort(key=lambda x: x['Influence'], reverse=True)
# print(BB)

# --- Load ub (User influence data) ---
ub = []
with open("new_ub.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row['Influenced Billboards'] = ast.literal_eval(row['Influenced Billboards'])
        ub.append(row)

# --- Load Dem (Demand values) ---
Dem = []
with open("Advertiser_LA1.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        Dem.append(float(row['Demand']))

# BillboardSlot class
class BillboardSlot:
    def __init__(self, idx):
        self.idx = idx
    def __repr__(self):
        return f"Slot({self.idx})"

# Convert BB to BillboardSlot objects using B_index
BS = [BillboardSlot(item['B_index']) for item in BB[:5000]]
# print("BS",BS)

In [2]:
# Influence function
def influence(S):
    influence_score = 0.0
    slot_ids = {s.idx if isinstance(s, BillboardSlot) else s for s in S}
    # print("slot_ids",slot_ids)
    for row in ub:
        influenced_billboards = row['Influenced Billboards']
        intersection_result = {int(key): value for key, value in influenced_billboards.items() if int(key) in slot_ids}
        product = 1.0
        # print("Intersection Result:",intersection_result)
        for value in intersection_result.values():
            # print("vlaue",value)
            
            product *= (1 - value)
            # print("product",product)
        influence_score += (1 - product)
        # print("influence_score",influence_score)
    return influence_score

In [3]:
def assign_top_influential(BS, Dem):
    used_slots = set()               # Track globally used slots
    Si_list = [set() for _ in Dem]   # List of selected slots per product

    index = 0  # Index into sorted BS list (assumed sorted by influence descending)

    for j in range(len(Dem)):
        demand = Dem[j]
        current_influence = 0.0

        while current_influence < demand and index < len(BS):
            slot = BS[index]
            index += 1

            if slot.idx in used_slots:
                continue  # Skip if slot already used for another product

            Si_list[j].add(slot.idx)
            current_influence = influence(Si_list[j])
            used_slots.add(slot.idx)

        # Optional: check if the demand was actually satisfied
        if current_influence < demand:
            print(f"Warning: Demand for product {j} not fully satisfied. Got {current_influence:.2f}, needed {demand:.2f}")

    return Si_list


In [4]:
# Run the allocation
result = assign_top_influential(BS, Dem)
total_influence = 0
# Total unique slots
all_selected = set().union(*result)
# Print per-product slot selections
for i, slots in enumerate(result):
    print(f"Product {i+1}: Selected Slots = {sorted(list(slots))}, Influence = {influence(slots):.2f}")
    total_influence += influence(slots)
# Calculate total unique slots and total influence
print("\nTotal Selected Slots (unique):", len(all_selected))
print("Total Combined Influence:", total_influence)
print("Execution time:", round(time.time() - start, 2), "seconds")


Product 1: Selected Slots = [1060, 4178], Influence = 31.43
Product 2: Selected Slots = [3715], Influence = 15.57
Product 3: Selected Slots = [2872], Influence = 14.71
Product 4: Selected Slots = [2974], Influence = 14.69
Product 5: Selected Slots = [621, 1760], Influence = 27.48
Product 6: Selected Slots = [2473, 2698], Influence = 25.10
Product 7: Selected Slots = [956, 2191], Influence = 17.85
Product 8: Selected Slots = [1443, 2289], Influence = 17.69
Product 9: Selected Slots = [713, 3133], Influence = 17.45
Product 10: Selected Slots = [1239, 1347], Influence = 17.20
Product 11: Selected Slots = [2684, 2873], Influence = 17.16
Product 12: Selected Slots = [377, 4367], Influence = 17.06
Product 13: Selected Slots = [891, 1523], Influence = 16.92
Product 14: Selected Slots = [66, 4330], Influence = 16.78
Product 15: Selected Slots = [3838, 4599], Influence = 19.49
Product 16: Selected Slots = [1070, 4055], Influence = 22.19
Product 17: Selected Slots = [3422, 3718], Influence = 16.

In [5]:
h = len(Dem)
count = 0

for j in range(h):
    if influence(result[j]) >= Dem[j]:
        count += 1

print("Number of products whose demands are satisfied:", count, "out of", h)


Number of products whose demands are satisfied: 100 out of 100
